In [1]:
import sys
sys.path.insert(1, "../Generation Automates")

In [2]:
from automata import *

# Génération des séquences de jonglage simple associées à des musiques

## Limitations

Évidemment, les solutions générées ne sont pas trop réalisables car elles demandent vite des lancers d'une hauteur très importante.

## Idée de tout ce qui suit

- Transformer une séquence de jonglage à partir d'un certain état en une musique
- Puis, pour transformer une musique en séquence de jonglage :
    - Pour chaque état de l'automate à b balles et de hauteur maximale h :
        - Parcourir en l'automate paresseusement en partant de cet état et en backtrackant lorsque la musique que l'on est en train de construire n'est diffère de celle fournie.

Avec ce procédé, on construit (je pense) toutes les séquences de jonglage correspondant à la musique voulue. Il faut aussi rajouter, au début de cette séquence, la suite de lancers pour construire l'état nécessaire au début de la séquence.

## Cas du jonglage multiplex et multihand - Rythmes et notes

### Automate

In [3]:
from sage.all import Combinations, OrderedSetPartitions, cartesian_product_iterator
import copy as cp

In [4]:
def InsertAtEachPosition(l, e):
    for i in range(0, len(l) + 1):
        l1 = l.copy()
        l1.insert(i, e)
        yield l1

def EmpOrderedSetPartitions(s, k):
    for p in OrderedSetPartitions(s, k):
        yield p
    for i in range(k - 1, 0, -1):
        P = OrderedSetPartitions(s, i)
        for p in P:
            L = [list(p)]
            for j in range(0, k - i):
                L1 = []
                for pl in L:
                    e = set()
                    l1 = list(InsertAtEachPosition(pl, e))
                    L1 = L1 + l1
                L = L1
            for l in L:
                yield l

In [5]:
class Automata():
    def __init__(self, balls, max_height, nb_hands):
        self.max_height = max_height
        self.balls = balls
        self.nb_hands = 2 # nb_hands
        self.initial = [[[] for i in range(max_height)] for h in range(nb_hands)]
        self.initial[0][0] = balls
    
    def transition(self, state, times):
        s = cp.deepcopy(state)
        r = []

        for h in range(self.nb_hands):
            r.append(s[h][0])
            del s[h][0]
            s[h].append(0)

        for h in range(self.nb_hands):
            htimes = times[h] # les lancers de la main h
            rh = r[h] # le nombre de balles dans la main h

            # ici, pour chaque main h, on veut une structure de la forme :
            # h' -> [liste de hauteurs de lancers]
            # on veut donc une liste qui à chaque main associe la liste
            # des hauteurs de balles qu'on lui envoi
            if rh >= 0:
                for hdest in range(self.nb_hands):
                    if rh < len(htimes[hdest]):
                        # si le nombre de balles restantes dans la main
                        # est inférieur au nombre de balles à lancer
                        # de h vers hdest
                        raise Exception("error")
                    else:
                        for t in htimes[hdest]:
                            s[hdest][t - 1] += 1
                        rh -= len(htimes[hdest])
            else:
                raise Exception("error")
        return s
    
    def possible_transitions(self, state):
        transitions = []
        
        if state[0][0] == 0:
            for c2 in Combinations(list(range(1, self.max_height + 1)), state[1][0]):
                for c2p in EmpOrderedSetPartitions(c2, 2):
                    try:
                        c = [[[], []], [list(x) for x in c2p]]
                        s = self.transition(state, c)
                        transitions.append((c, s))
                    except Exception as e:
                        if e.args[0] != "error":
                            raise e
        elif state[1][0] == 0:
            for c1 in Combinations(list(range(1, self.max_height + 1)), state[0][0]):
                for c1p in EmpOrderedSetPartitions(c1, 2):
                    try:
                        c = [[list(x) for x in c1p], [[], []]]
                        s = self.transition(state, c)
                        transitions.append((c, s))
                    except Exception as e:
                        if e.args[0] != "error":
                            raise e
        elif state[0][0] > 0 and state[1][0] > 0:
            for c1 in Combinations(list(range(1, self.max_height + 1)), state[0][0]):
                for c2 in Combinations(list(range(1, self.max_height + 1)), state[1][0]):
                    for c1p in EmpOrderedSetPartitions(list(range(1, self.max_height + 1)), 2):
                        for c2p in EmpOrderedSetPartitions(list(range(1, self.max_height + 1)), 2):
                            try:
                                lc1 = [list(x) for x in c1p]
                                lc2 = [list(x) for x in c2p]
                                c = [lc1, lc2]
                                s = self.transition(state, c)
                                transitions.append((c, s))
                            except Exception as e:
                                if e.args[0] != "error":
                                    raise e
        
        return transitions

    def state_str(self, s : list[list[int]]):
        s1 = [str(x) for x in s]
        return "".join(s1)

In [6]:
def print_transitions(transitions):
    for throw, state in transitions:
        print(str(throw), "->", str(state))

In [7]:
a = Automata(3, 5, 2)

trs = a.possible_transitions([[1, 0, 1, 0, 0], [0, 1, 0, 0, 0]])

print_transitions(trs)

[[[], [1]], [[], []]] -> [[0, 1, 0, 0, 0], [2, 0, 0, 0, 0]]
[[[1], []], [[], []]] -> [[1, 1, 0, 0, 0], [1, 0, 0, 0, 0]]
[[[], [2]], [[], []]] -> [[0, 1, 0, 0, 0], [1, 1, 0, 0, 0]]
[[[2], []], [[], []]] -> [[0, 2, 0, 0, 0], [1, 0, 0, 0, 0]]
[[[], [3]], [[], []]] -> [[0, 1, 0, 0, 0], [1, 0, 1, 0, 0]]
[[[3], []], [[], []]] -> [[0, 1, 1, 0, 0], [1, 0, 0, 0, 0]]
[[[], [4]], [[], []]] -> [[0, 1, 0, 0, 0], [1, 0, 0, 1, 0]]
[[[4], []], [[], []]] -> [[0, 1, 0, 1, 0], [1, 0, 0, 0, 0]]
[[[], [5]], [[], []]] -> [[0, 1, 0, 0, 0], [1, 0, 0, 0, 1]]
[[[5], []], [[], []]] -> [[0, 1, 0, 0, 1], [1, 0, 0, 0, 0]]


In [10]:
class NotesAutomata():
    def __init__(self, balls, max_height, nb_hands):
        self.max_height = max_height
        self.balls = balls
        self.nb_hands = 2 # nb_hands
        self.initial = [[[] for i in range(max_height)] for h in range(nb_hands)]
        self.initial[0][0] = balls
    
    def transition(self, state, throw):
        new_state = [cp.deepcopy(hand[1:]) + [[]] for hand in state]
        for src_hand in range(self.nb_hands):
            for dst_hand in range(self.nb_hands):
                for t, ball in throw[src_hand][dst_hand]:
                    if t == None:
                        new_state[dst_hand][0].append(ball)
                    else:
                        new_state[dst_hand][t - 1].append(ball)
                    
        return new_state
    
    def all_throws_ball(self, src_hand, ball, none):
        throws = []

        for dst_hand in range(self.nb_hands):
            for height in range(1, self.max_height + 1):
                t = [[[] for dst_hand1 in range(self.nb_hands)] 
                         for src_hand1 in range(self.nb_hands)]
                t[src_hand][dst_hand] = [(height, ball)]
                throws.append(t)
        
        if none:
            t = [[[] for dst_hand1 in range(self.nb_hands)] 
                     for src_hand1 in range(self.nb_hands)]
            t[src_hand][src_hand] = [(None, ball)]
            throws.append(t)

        return throws
    
    def combine_throw_list(self, throw_list):
        throw = []

        for src_hand in range(self.nb_hands):
            src_throws = []
            for dst_hand in range(self.nb_hands):
                l = []
                for t in throw_list:
                    l += t[src_hand][dst_hand]
                src_throws.append(tuple(l))
            throw.append(tuple(src_throws))

        return tuple(throw)
    
    def mix_ball_throws(self, ball_throws):
        throws = set()
        
        for c in cartesian_product_iterator([ball_throws[k] for k in ball_throws]):
            t = self.combine_throw_list(list(c))
            throws.add(t)

        return throws
    
    def all_throws(self, hands_content, none):
        ball_throws = {}

        for i in range(self.nb_hands):
            balls = hands_content[i]
            for ball in balls:
                ball_throws[ball] = self.all_throws_ball(i, ball, none)

        throws = self.mix_ball_throws(ball_throws)

        return throws
    
    def possible_transitions(self, state, none):
        hands_content = [hand[0] for hand in state]
        possible_throws = self.all_throws(hands_content, none)
        transitions = []

        for throw in possible_throws:
            s = a.transition(state, throw)
            transitions.append((throw, s))

        return transitions

    def state_str(self, s : list[list[int]]):
        s1 = [str(x) for x in s]
        return "".join(s1)
    
    def state_tuple(self, s):
        return tuple([tuple([tuple(t) for t in h]) for h in s])

In [298]:
class AltNotesAutomata():
    def __init__(self, balls, max_height, nb_hands):
        self.max_height = max_height
        self.balls = balls
        self.nb_hands = 2 # nb_hands
        self.initial = [[[] for i in range(max_height)] for h in range(nb_hands)]
        self.initial[0][0] = balls
    
    def transition(self, state, throw):
        new_state = [cp.deepcopy(hand[1:]) + [[]] for hand in state]
        for src_hand in range(self.nb_hands):
            for dst_hand in range(self.nb_hands):
                for t, ball in throw[src_hand][dst_hand]:
                    if t == None:
                        new_state[dst_hand][0].append(ball)
                    else:
                        new_state[dst_hand][t - 1].append(ball)
                    
        return new_state
    
    def all_throws_ball(self, src_hand, ball, none):
        throws = []

        for dst_hand in range(self.nb_hands):
            for height in range(1, self.max_height + 1):
                t = [[[] for dst_hand1 in range(self.nb_hands)] 
                         for src_hand1 in range(self.nb_hands)]
                t[src_hand][dst_hand] = [(height, ball)]
                throws.append(t)
        
        if none:
            t = [[[] for dst_hand1 in range(self.nb_hands)] 
                     for src_hand1 in range(self.nb_hands)]
            t[src_hand][src_hand] = [(None, ball)]
            throws.append(t)

        return throws
    
    def combine_throw_list(self, throw_list):
        throw = []

        for src_hand in range(self.nb_hands):
            src_throws = []
            for dst_hand in range(self.nb_hands):
                l = []
                for t in throw_list:
                    l += t[src_hand][dst_hand]
                src_throws.append(tuple(l))
            throw.append(tuple(src_throws))

        return tuple(throw)
    
    def mix_ball_throws(self, ball_throws):
        throws = set()
        
        for c in cartesian_product_iterator([ball_throws[k] for k in ball_throws]):
            t = self.combine_throw_list(list(c))
            throws.add(t)

        return throws
    
    def all_throws(self, hands_content, none):
        ball_throws = {}

        for i in range(self.nb_hands):
            balls = hands_content[i]
            for ball in balls:
                ball_throws[ball] = self.all_throws_ball(i, ball, none)

        throws = self.mix_ball_throws(ball_throws)

        return throws
    
    def possible_transitions(self, state, t):
        transitions = []
        hands_content = [hand[0] for hand in state]
        try:
            for i in range(2, self.max_height + 1):
                src_hand = t % self.nb_hands
                dst_hand = (t + i) % self.nb_hands
                throw = [[[] for d in range(self.nb_hands)] 
                                  for s in range(self.nb_hands)]
                ball = hands_content[src_hand][0]
                throw[src_hand][dst_hand].append((i, ball))
                for s in range(self.nb_hands):
                    for b in hands_content[s]:
                        if b != ball:
                            throw[s][s].append((None, b))
                throw_tpl = tuple([tuple([
                    tuple(throw[s][d]) for d in range(self.nb_hands)]) 
                                       for s in range(self.nb_hands)])
                next_state = a.transition(state, throw_tpl)
                transitions.append((throw_tpl, next_state))
                
            # Ball stay in hand
            throw = [[[] for d in range(self.nb_hands)]
                         for s in range(self.nb_hands)]
            for s in range(self.nb_hands):
                for b in hands_content[s]:
                    throw[s][s].append((None, b))
            throw_tpl = tuple([tuple([
                tuple(throw[s][d]) for d in range(self.nb_hands)]) 
                                   for s in range(self.nb_hands)])
            next_state = a.transition(state, throw_tpl)
            transitions.append((throw_tpl, next_state))
            
        except IndexError as e:
            print("main vide !")

        return transitions

    def state_str(self, s : list[list[int]]):
        s1 = [str(x) for x in s]
        return "".join(s1)
    
    def state_tuple(self, s):
        return tuple([tuple([tuple(t) for t in h]) for h in s])

In [299]:
a = NotesAutomata(["do", "ré", "mi"], 5, 2)

In [300]:
throws = a.all_throws([["do", "ré"], ["mi"]], False)

In [301]:
throws

{((((1, 'do'), (2, 'ré')), ()), ((), ((5, 'mi'),))),
 ((((2, 'ré'),), ((3, 'do'),)), ((), ((3, 'mi'),))),
 ((((4, 'ré'),), ((4, 'do'),)), (((5, 'mi'),), ())),
 ((((4, 'do'), (2, 'ré')), ()), ((), ((4, 'mi'),))),
 ((((5, 'do'),), ((4, 'ré'),)), ((), ((2, 'mi'),))),
 ((((3, 'ré'),), ((5, 'do'),)), (((5, 'mi'),), ())),
 ((((2, 'do'), (2, 'ré')), ()), (((2, 'mi'),), ())),
 (((), ((1, 'do'), (3, 'ré'))), (((1, 'mi'),), ())),
 (((), ((3, 'do'), (1, 'ré'))), (((3, 'mi'),), ())),
 ((((4, 'do'),), ((3, 'ré'),)), ((), ((4, 'mi'),))),
 ((((2, 'do'), (5, 'ré')), ()), ((), ((3, 'mi'),))),
 ((((2, 'do'),), ((3, 'ré'),)), (((1, 'mi'),), ())),
 (((), ((4, 'do'), (5, 'ré'))), (((5, 'mi'),), ())),
 ((((1, 'do'),), ((3, 'ré'),)), (((3, 'mi'),), ())),
 ((((2, 'do'),), ((1, 'ré'),)), ((), ((2, 'mi'),))),
 ((((3, 'ré'),), ((2, 'do'),)), (((3, 'mi'),), ())),
 ((((2, 'do'),), ((5, 'ré'),)), (((4, 'mi'),), ())),
 ((((1, 'do'),), ((1, 'ré'),)), ((), ((5, 'mi'),))),
 ((((5, 'ré'),), ((1, 'do'),)), (((1, 'mi'),),

In [302]:
len(throws)

1000

In [303]:
a = AltNotesAutomata(["do", "ré", "mi"], 5, 2)

trs1 = a.possible_transitions([[["do"], [], ["ré"], [], []], [[], ["mi"], [], [], []]], 0)
print_transitions(trs1)
print("-" * 20)

trs2 = a.possible_transitions([[[], ['ré'], [], [], []], [['mi'], [], ['do'], [], []]], 1)
print_transitions(trs2)
print("-" * 20)

trs3 = a.possible_transitions([[['ré'], [], ['mi'], [], []], [[], ['do'], [], [], []]], 2)
print_transitions(trs3)
print("-" * 20)

trs4 = a.possible_transitions([[[], ['mi'], [], [], []], [['do'], [], ['ré'], [], []]], 3)
print_transitions(trs4)
print("-" * 20)

trs5 = a.possible_transitions([[['mi'], [], ['do'], [], []], [[], ['ré'], [], [], []]], 4)
print_transitions(trs5)
print("-" * 20)

trs6 = a.possible_transitions([[[], ['do'], [], [], []], [['ré'], [], ['mi'], [], []]], 5)
print_transitions(trs6)
print("-" * 20)

((((2, 'do'),), ()), ((), ())) -> [[[], ['ré', 'do'], [], [], []], [['mi'], [], [], [], []]]
(((), ((3, 'do'),)), ((), ())) -> [[[], ['ré'], [], [], []], [['mi'], [], ['do'], [], []]]
((((4, 'do'),), ()), ((), ())) -> [[[], ['ré'], [], ['do'], []], [['mi'], [], [], [], []]]
(((), ((5, 'do'),)), ((), ())) -> [[[], ['ré'], [], [], []], [['mi'], [], [], [], ['do']]]
((((None, 'do'),), ()), ((), ())) -> [[['do'], ['ré'], [], [], []], [['mi'], [], [], [], []]]
--------------------
(((), ()), ((), ((2, 'mi'),))) -> [[['ré'], [], [], [], []], [[], ['do', 'mi'], [], [], []]]
(((), ()), (((3, 'mi'),), ())) -> [[['ré'], [], ['mi'], [], []], [[], ['do'], [], [], []]]
(((), ()), ((), ((4, 'mi'),))) -> [[['ré'], [], [], [], []], [[], ['do'], [], ['mi'], []]]
(((), ()), (((5, 'mi'),), ())) -> [[['ré'], [], [], [], ['mi']], [[], ['do'], [], [], []]]
(((), ()), ((), ((None, 'mi'),))) -> [[['ré'], [], [], [], []], [['mi'], ['do'], [], [], []]]
--------------------
((((2, 'ré'),), ()), ((), ())) -> [[[]

In [304]:
a = AltNotesAutomata(["do", "ré", "mi"], 5, 2)

trs = a.possible_transitions([[["do", "ré"], [], [], [], []], [[], [], ["mi"], [], []]], 0)

print_transitions(trs)

((((2, 'do'), (None, 'ré')), ()), ((), ())) -> [[['ré'], ['do'], [], [], []], [[], ['mi'], [], [], []]]
((((None, 'ré'),), ((3, 'do'),)), ((), ())) -> [[['ré'], [], [], [], []], [[], ['mi'], ['do'], [], []]]
((((4, 'do'), (None, 'ré')), ()), ((), ())) -> [[['ré'], [], [], ['do'], []], [[], ['mi'], [], [], []]]
((((None, 'ré'),), ((5, 'do'),)), ((), ())) -> [[['ré'], [], [], [], []], [[], ['mi'], [], [], ['do']]]
((((None, 'do'), (None, 'ré')), ()), ((), ())) -> [[['do', 'ré'], [], [], [], []], [[], ['mi'], [], [], []]]


### Séquence -> Musique

In [305]:
def balls_still_flying(s):
    for hand in s:
        for t in hand:
            if len(t) > 0:
                return True
    return False

In [306]:
def seq_to_music(a, initial, seq):
    s = initial
    time_flying = {k: 0 for k in a.balls}
    not_flying = set()
    l = []
    for throw in seq:
        notes = []
        for b in time_flying:
            time_flying[b] += 1
        for hand in s:
            if hand[0] != []:
                for b in hand[0]:
                    if b not in not_flying:
                        notes.append((b, time_flying[b]))
                        time_flying[b] = 0
        if notes != []:
            l.append(notes)
        for src_hand in throw:
            for dst_hand in src_hand:
                for h, b in dst_hand:
                    if h == None:
                        not_flying.add(b)
                    elif b in not_flying:
                        not_flying = not_flying - {b}
        s = a.transition(s, throw)
    while balls_still_flying(s):
        notes = []
        for b in time_flying:
            time_flying[b] += 1
        for hand in s:
            if hand[0] != []:
                for b in hand[0]:
                    if b not in not_flying:
                        notes.append((b, time_flying[b]))
                        time_flying[b] = 0
        if notes != []:
            l.append(notes)
        s = [hand[1:] for hand in s]
    return l

In [307]:
a = NotesAutomata(["do", "ré", "mi"], 5, 2)
seq_to_music(a, 
             [[["do", "ré", "mi"], [], [], [], []], [[], [], [], [], []]], 
             [
                 (((), ((1, 'do'),)), ((), ())),
                 (((), ((3, 'ré'),)), (((1, 'do'),), ())),
                 ((((3, 'mi'),), ((1, 'do'),)), ((), ()))
             ])

[[('do', 1), ('ré', 1), ('mi', 1)],
 [('do', 1)],
 [('do', 1)],
 [('do', 1)],
 [('ré', 4)],
 [('mi', 5)]]

In [308]:
a.transition([[["do", "ré", "mi"], [], [], [], []], [[], [], [], [], []]], (((), ((1, 'do'),)), ((), ())))

[[[], [], [], [], []], [['do'], [], [], [], []]]

In [309]:
a = NotesAutomata(["do"], 5, 2)

a.transition([[["do"], [], [], [], []], [[], [], [], [], []]], (((), ((5, 'do'),)), ((), ())))

seq_to_music(a,
             [[["do"], [], [], [], []], [[], [], [], [], []]], 
             [
                 (((), ((5, 'do'),)), ((), ())),
                 (((), ()), ((), ())),
                 (((), ()), ((), ())),
                 (((), ()), ((), ())),
                 (((), ()), ((), ())),
                 (((), ()), ((), ((None, 'do'),))),
                 (((), ()), ((), ((None, 'do'),))),
             ])

[[('do', 1)], [('do', 5)]]

In [310]:
a = AltNotesAutomata(["do", "ré", "mi"], 5, 2)

seq_to_music(a,
             [[['do'], [], ['ré'], [], []], [[], ['mi'], [], [], []]],
             [
                 (((), ((3, 'do'),)), ((), ())),
                 (((), ()), (((3, 'mi'),), ())),
                 (((), ((3, 'ré'),)), ((), ())),
                 (((), ()), (((3, 'do'),), ())),
                 (((), ((3, 'mi'),)), ((), ())),
                 (((), ()), (((3, 'ré'),), ()))
             ])

[[('do', 1)],
 [('mi', 2)],
 [('ré', 3)],
 [('do', 3)],
 [('mi', 3)],
 [('ré', 3)],
 [('do', 3)],
 [('mi', 3)],
 [('ré', 3)]]

### Musique -> Séquence

In [311]:
from sage.all import Subsets

In [312]:
[list(s) for s in Subsets(["do", "ré", "mi"])]

[[],
 ['do'],
 ['ré'],
 ['mi'],
 ['do', 'ré'],
 ['do', 'mi'],
 ['ré', 'mi'],
 ['do', 'ré', 'mi']]

In [313]:
def land_possibilities(balls, nb_hands):
    subsets = Subsets(balls)
    P = [[set() for h in range(nb_hands)]]
    for s in subsets:
        for p in EmpOrderedSetPartitions(s, nb_hands):
            P.append(p)
    return P

In [314]:
land_possibilities(["do", "ré", "mi"], 2)

[[set(), set()],
 [set(), {'do'}],
 [{'do'}, set()],
 [set(), {'ré'}],
 [{'ré'}, set()],
 [set(), {'mi'}],
 [{'mi'}, set()],
 [{'do'}, {'ré'}],
 [{'ré'}, {'do'}],
 [set(), {'do', 'ré'}],
 [{'do', 'ré'}, set()],
 [{'do'}, {'mi'}],
 [{'mi'}, {'do'}],
 [set(), {'do', 'mi'}],
 [{'do', 'mi'}, set()],
 [{'ré'}, {'mi'}],
 [{'mi'}, {'ré'}],
 [set(), {'ré', 'mi'}],
 [{'ré', 'mi'}, set()],
 [{'do', 'ré'}, {'mi'}],
 [{'do', 'mi'}, {'ré'}],
 [{'mi', 'ré'}, {'do'}],
 [{'do'}, {'mi', 'ré'}],
 [{'ré'}, {'do', 'mi'}],
 [{'mi'}, {'do', 'ré'}],
 [set(), {'do', 'ré', 'mi'}],
 [{'do', 'ré', 'mi'}, set()]]

In [315]:
a = set()
a = a.union({2, 3})
a

{2, 3}

In [316]:
def rmv_possibilities(landp, balls):
    landp1 = []
    for lp in landp:
        elim = False
        for h in lp:
            for b in h:
                if b in balls:
                    elim = True
                    break
            if elim:
                break
        if not elim:
            landp1.append(lp)
    return landp1

In [323]:
def states(balls, max_height, nb_hands):
    landpos = land_possibilities(balls, nb_hands)
    if max_height == 0:
        return [[] for h in range(nb_hands)]
    elif max_height > 0:
        def states_rec(landp, i, l):
            if len(landp) == 1 and i == -1:
                yield l.copy()
            elif len(landp) >= 1 and i >= 0:
                for lp in landp:
                    l1 = cp.deepcopy(l)
#                     print(l)
#                     print(lp)
                    for h in range(nb_hands):
                        l1[h] = [list(lp[h])] + l1[h]
#                     print(l1)
#                     print("-" * 20)
                    balls = set()
                    for h in range(nb_hands):
                        balls = balls.union(lp[h])
                    next_landp = rmv_possibilities(landp, balls)
                    for e in states_rec(next_landp, i - 1, l1):
                        yield e
        return states_rec(landpos, max_height - 1, [[] for i in range(nb_hands)])

In [324]:
lp = land_possibilities(["do"], 2)

In [325]:
rmv_possibilities([[set(), set()]], {})

[[set(), set()]]

In [326]:
list(states(["do"], 5, 2))

[[[[], [], [], [], []], [['do'], [], [], [], []]],
 [[['do'], [], [], [], []], [[], [], [], [], []]],
 [[[], [], [], [], []], [[], ['do'], [], [], []]],
 [[[], ['do'], [], [], []], [[], [], [], [], []]],
 [[[], [], [], [], []], [[], [], ['do'], [], []]],
 [[[], [], ['do'], [], []], [[], [], [], [], []]],
 [[[], [], [], [], []], [[], [], [], ['do'], []]],
 [[[], [], [], ['do'], []], [[], [], [], [], []]],
 [[[], [], [], [], []], [[], [], [], [], ['do']]],
 [[[], [], [], [], ['do']], [[], [], [], [], []]]]

In [327]:
list(states(["do", "ré", "mi"], 5, 2))

[[[['do', 'ré'], [], [], [], []], [['mi'], [], [], [], []]],
 [[['do', 'mi'], [], [], [], []], [['ré'], [], [], [], []]],
 [[['ré', 'mi'], [], [], [], []], [['do'], [], [], [], []]],
 [[['do'], [], [], [], []], [['ré', 'mi'], [], [], [], []]],
 [[['ré'], [], [], [], []], [['do', 'mi'], [], [], [], []]],
 [[['mi'], [], [], [], []], [['do', 'ré'], [], [], [], []]],
 [[[], [], [], [], []], [['do', 'ré', 'mi'], [], [], [], []]],
 [[['do', 'ré', 'mi'], [], [], [], []], [[], [], [], [], []]],
 [[['ré'], [], [], [], []], [['mi'], ['do'], [], [], []]],
 [[['mi'], [], [], [], []], [['ré'], ['do'], [], [], []]],
 [[[], [], [], [], []], [['ré', 'mi'], ['do'], [], [], []]],
 [[['ré', 'mi'], [], [], [], []], [[], ['do'], [], [], []]],
 [[['ré'], ['do'], [], [], []], [['mi'], [], [], [], []]],
 [[['mi'], ['do'], [], [], []], [['ré'], [], [], [], []]],
 [[[], ['do'], [], [], []], [['ré', 'mi'], [], [], [], []]],
 [[['ré', 'mi'], ['do'], [], [], []], [[], [], [], [], []]],
 [[['do'], [], [], [], []], 

In [328]:
a = NotesAutomata(["do", "ré", "mi"], 5, 2)

a.state_tuple([[["do"], [], [], [], ["ré"]], [[], [], ["mi"], [], []]])

((('do',), (), (), (), ('ré',)), ((), (), ('mi',), (), ()))

In [329]:
def only_None(throw):
    for src_hand in throw:
        for dst_hand in src_hand:
            for h, b in dst_hand:
                if h != None:
                    return False
    return True

In [330]:
def throw_all_balls_None(a, throw):
    balls = set(a.balls)
    for src_hand in throw:
        for dst_hand in src_hand:
            for h, b in dst_hand:
                if h is None and b in balls:
                    balls.remove(b)
    return (len(balls) == 0)

In [331]:
a = AltNotesAutomata(["do", "ré", "mi"], 5, 2)

throw_all_balls_None(a, ((((None, 'ré'), (None, 'do'),), ()), ((), ((None, 'mi'),))))

True

In [332]:
def alternates(seq):
    if len(seq) == 0:
        return True
    
    begin_hand = None
    for src_hand in range(len(seq[0])):
        for dst_hand in range(len(seq[0][src_hand])):
            for h, b in seq[0][src_hand][dst_hand]:
                if h != None and begin_hand is None:
                    begin_hand = src_hand
                elif h != None and begin_hand is not None:
                    return False
                
    if begin_hand is None:
        raise Exception("No first ball")
        
    none_hand = begin_hand
    
    for i in range(1, len(seq)):
        for dst_hand in range(len(seq[i][none_hand])):
            for h, b in seq[i][none_hand][dst_hand]:
                if h is not None:
                    return False
        none_hand = 1 - none_hand
    return True

In [333]:
alternates([
    ((((1, 'do'), (None, 'ré')), ()), (((None, 'mi'),), ())),
    ((((None, 'do'), (None, 'ré')), ()), (((2, 'mi'),), ())),
])

True

In [362]:
def music_to_seq_rec(a, music, possibilities, initial, state, seq, 
                     t, cnt_all_None, max_all_None):
    m = seq_to_music(a, initial, seq)
    equal = True
    if len(m) > len(music):
        raise Exception("backtrack")
#     if not alternates(seq):
#         raise Exception("backtrack")
    for i in range(len(m)):
        if not set(m[i]).issubset(set(music[i])):
            raise Exception("backtrack")
    if len(m) == len(music):
        for i in range(len(music)):
            if set(m[i]) != set(music[i]):
                equal = False
    else:
        equal = False
    if equal:
        initial_tpl = a.state_tuple(initial)
#         print("Tuple :", initial_tpl)
        if initial_tpl in possibilities:
            possibilities[initial_tpl].append(seq)
        else:
            possibilities[initial_tpl] = [seq]
        return
    for throw, next_state in a.possible_transitions(state, t):
        try:
            if throw_all_balls_None(a, throw) and cnt_all_None > 0:
                music_to_seq_rec(a, music, possibilities, initial, 
                                 next_state, seq + [throw], t + 1, 
                                 cnt_all_None - 1, max_all_None)
            elif not throw_all_balls_None(a, throw):
                music_to_seq_rec(a, music, possibilities, initial, 
                                 next_state, seq + [throw], t + 1, 
                                 max_all_None, max_all_None)
        except Exception as e:
            if e.args[0] != "backtrack":
                raise e

In [386]:
def music_to_seq_initial(a, initial, music, max_all_None):
    possibilities = dict()
    music_to_seq_rec(a, music, possibilities, initial, initial, [], 0,
                     max_all_None, max_all_None)
    return possibilities

In [387]:
def music_to_seq(a, music, max_all_None):
    possibilities = dict()
    for initial in states(set(a.balls), a.max_height, a.nb_hands):
        try:
#             print("try :", initial)
            music_to_seq_rec(a, music, possibilities, initial, initial, [], 0, 
                             max_all_None, max_all_None)
        except Exception as e:
            if e.args[0] != "backtrack":
                raise e
    return possibilities

In [388]:
list(states(set(["do"]), 5, 2))

[[[[], [], [], [], []], [['do'], [], [], [], []]],
 [[['do'], [], [], [], []], [[], [], [], [], []]],
 [[[], [], [], [], []], [[], ['do'], [], [], []]],
 [[[], ['do'], [], [], []], [[], [], [], [], []]],
 [[[], [], [], [], []], [[], [], ['do'], [], []]],
 [[[], [], ['do'], [], []], [[], [], [], [], []]],
 [[[], [], [], [], []], [[], [], [], ['do'], []]],
 [[[], [], [], ['do'], []], [[], [], [], [], []]],
 [[[], [], [], [], []], [[], [], [], [], ['do']]],
 [[[], [], [], [], ['do']], [[], [], [], [], []]]]

In [389]:
a = NotesAutomata(["do"], 5, 2)

In [390]:
seq_to_music(a,
             [[["do"], [], [], [], []], [[], [], [], [], []]], 
             [
                 (((), ((1, 'do'),)), ((), ()))
             ])

[[('do', 1)], [('do', 1)]]

In [391]:
seq_to_music(a,
             [[["do"], [], [], [], []], [[], [], [], [], []]], 
             [
                 (((), ((2, 'do'),)), ((), ()))
             ])

[[('do', 1)], [('do', 2)]]

In [392]:
init = list(states(["do", "ré", "mi"], 5, 2))
for s in init:
    for h in s:
        if len(h) < 5:
            print(s)

In [393]:
[[['mi', 'ré'], [], [], []], [['do'], [], [], []]] in init

False

In [394]:
[[['ré', 'mi'], [], [], [], []], [['do'], [], [], [], []]] in init

True

In [395]:
[[['do', 'ré'], [], [], [], []], [[], [], [], [], []]] in list(states(["do", "ré"], 5, 2))

True

In [396]:
def pp_throw(indent, throw):
    strs = []
    for src_hand in range(len(throw)):
        for dst_hand in range(len(throw[src_hand])):
            for h, b in throw[src_hand][dst_hand]:
                strs.append("{} : {} -- {} --> {}".format(b, src_hand, h, dst_hand))
    print(indent + ", ".join(strs))

In [397]:
def pp_seq(indent, seq):
    for throw in seq:
        pp_throw(indent, throw)

In [440]:
def pp_propositions(d):
    for k in d.keys():
        print(str(k) + " : ")
        indent = "    " # "".join([" " for e in range(len(str(k)) + 3)])
#         pp_throw("", d[k][0][0])
#         print(d[k][0])
#         if len(d[k][0]) > 1:
#             pp_seq(indent, d[k][0][1:])
#         print(indent + "-" * 20)
        for i in range(0, len(d[k])):
            pp_seq(indent, d[k][i])
            print(indent + "-" * 20)

In [441]:
a = NotesAutomata(["do"], 5, 2)
p = music_to_seq(a, [[("do", 1)], [("do", 1)], [("do", 1)]], 1)

In [442]:
pp_propositions(p)

(((), (), (), (), ()), (('do',), (), (), (), ())) : 
    do : 1 -- 1 --> 1
    do : 1 -- 1 --> 1
    --------------------
    do : 1 -- 1 --> 1
    do : 1 -- 1 --> 0
    --------------------
    do : 1 -- 1 --> 0
    do : 0 -- 1 --> 0
    --------------------
    do : 1 -- 1 --> 0
    do : 0 -- 1 --> 1
    --------------------
((('do',), (), (), (), ()), ((), (), (), (), ())) : 
    do : 0 -- 1 --> 0
    do : 0 -- 1 --> 0
    --------------------
    do : 0 -- 1 --> 0
    do : 0 -- 1 --> 1
    --------------------
    do : 0 -- 1 --> 1
    do : 1 -- 1 --> 1
    --------------------
    do : 0 -- 1 --> 1
    do : 1 -- 1 --> 0
    --------------------


In [443]:
a = NotesAutomata(["do", "ré", "mi"], 5, 2)

p = music_to_seq(a, [[("do", 1), ("ré", 1), ("mi", 1)], 
                     [("do", 1)], 
                     [("ré", 2), ("mi", 2), ("do", 1)]], 1)
pp_propositions(p)

((('do', 'ré'), (), (), (), ()), (('mi',), (), (), (), ())) : 
    do : 0 -- 1 --> 0, ré : 0 -- 2 --> 1, mi : 1 -- 2 --> 1
    do : 0 -- 1 --> 0
    --------------------
    do : 0 -- 1 --> 0, ré : 0 -- 2 --> 1, mi : 1 -- 2 --> 1
    do : 0 -- 1 --> 1
    --------------------
    do : 0 -- 1 --> 0, ré : 0 -- 2 --> 1, mi : 1 -- 2 --> 0
    do : 0 -- 1 --> 0
    --------------------
    do : 0 -- 1 --> 0, ré : 0 -- 2 --> 1, mi : 1 -- 2 --> 0
    do : 0 -- 1 --> 1
    --------------------
    do : 0 -- 1 --> 0, ré : 0 -- 2 --> 0, mi : 1 -- 2 --> 1
    do : 0 -- 1 --> 0
    --------------------
    do : 0 -- 1 --> 0, ré : 0 -- 2 --> 0, mi : 1 -- 2 --> 1
    do : 0 -- 1 --> 1
    --------------------
    ré : 0 -- 2 --> 0, do : 0 -- 1 --> 1, mi : 1 -- 2 --> 1
    do : 1 -- 1 --> 1
    --------------------
    ré : 0 -- 2 --> 0, do : 0 -- 1 --> 1, mi : 1 -- 2 --> 1
    do : 1 -- 1 --> 0
    --------------------
    do : 0 -- 1 --> 0, ré : 0 -- 2 --> 0, mi : 1 -- 2 --> 0
    do : 0 -- 1 --> 0

In [444]:
a = AltNotesAutomata(["do", "ré", "mi"], 5, 2)

trs1 = a.possible_transitions([[['do', 'ré'], [], [], [], []], [['mi'], [], [], [], []]], 0)
print_transitions(trs1)
print("-" * 20)

trs2 = a.possible_transitions([[['ré'], [], [], [], []], [['mi'], [], ['do'], [], []]], 1)
print_transitions(trs2)
print("-" * 20)

trs3 = a.possible_transitions([[['ré'], [], [], [], []], [['mi'], ['do'], [], [], []]], 2)
print_transitions(trs3)
print("-" * 20)

trs4 = a.possible_transitions([[['ré'], [], [], [], []], [['do', 'mi'], [], [], [], []]], 3)
print_transitions(trs4)
print("-" * 20)

((((2, 'do'), (None, 'ré')), ()), ((), ((None, 'mi'),))) -> [[['ré'], ['do'], [], [], []], [['mi'], [], [], [], []]]
((((None, 'ré'),), ((3, 'do'),)), ((), ((None, 'mi'),))) -> [[['ré'], [], [], [], []], [['mi'], [], ['do'], [], []]]
((((4, 'do'), (None, 'ré')), ()), ((), ((None, 'mi'),))) -> [[['ré'], [], [], ['do'], []], [['mi'], [], [], [], []]]
((((None, 'ré'),), ((5, 'do'),)), ((), ((None, 'mi'),))) -> [[['ré'], [], [], [], []], [['mi'], [], [], [], ['do']]]
((((None, 'do'), (None, 'ré')), ()), ((), ((None, 'mi'),))) -> [[['do', 'ré'], [], [], [], []], [['mi'], [], [], [], []]]
--------------------
((((None, 'ré'),), ()), ((), ((2, 'mi'),))) -> [[['ré'], [], [], [], []], [[], ['do', 'mi'], [], [], []]]
((((None, 'ré'),), ()), (((3, 'mi'),), ())) -> [[['ré'], [], ['mi'], [], []], [[], ['do'], [], [], []]]
((((None, 'ré'),), ()), ((), ((4, 'mi'),))) -> [[['ré'], [], [], [], []], [[], ['do'], [], ['mi'], []]]
((((None, 'ré'),), ()), (((5, 'mi'),), ())) -> [[['ré'], [], [], [], ['mi']

In [445]:
a = AltNotesAutomata(["do", "ré", "mi"], 5, 2)

print_transitions(
    a.possible_transitions(
        [[['do'], [], ['ré'], [], []], [[], ['mi'], [], [], []]], 0))

seq_to_music(a,
             [[['do', 'ré'], [], [], [], []], [['mi'], [], [], [], []]],
             [
                 ((((None, 'ré'),), ((3, 'do'),)), ((), ((None, 'mi'),))),
                 ((((None, 'ré'),), ()), ((), ((None, 'mi'),))),
                 ((((None, 'ré'),), ()), ((), ((None, 'mi'),))),
                 ((((None, 'ré'),), ()), (((3, 'do'),), ((None, 'mi'),)))
             ])

((((2, 'do'),), ()), ((), ())) -> [[[], ['ré', 'do'], [], [], []], [['mi'], [], [], [], []]]
(((), ((3, 'do'),)), ((), ())) -> [[[], ['ré'], [], [], []], [['mi'], [], ['do'], [], []]]
((((4, 'do'),), ()), ((), ())) -> [[[], ['ré'], [], ['do'], []], [['mi'], [], [], [], []]]
(((), ((5, 'do'),)), ((), ())) -> [[[], ['ré'], [], [], []], [['mi'], [], [], [], ['do']]]
((((None, 'do'),), ()), ((), ())) -> [[['do'], ['ré'], [], [], []], [['mi'], [], [], [], []]]


[[('do', 1), ('ré', 1), ('mi', 1)], [('do', 3)], [('do', 3)]]

In [446]:
a = AltNotesAutomata(["do", "ré", "mi"], 5, 2)

throw_all_balls_None(a, ((((None, 'ré'), (None, 'do'),), ()), ((), ((None, 'mi'),))))

True

In [454]:
a = AltNotesAutomata(["do", "ré", "mi"], 25, 2)

p = music_to_seq(a, [
    [('do', 1), ('ré', 1), ('mi', 1)],
    [('do', 3)],
    [('do', 3)],
    [('do', 3)],
    [('ré', 12)],
    [('mi', 15)],
    [('ré', 9)],
    [('do', 18)],
    [('mi', 15)]
], 20)

pp_propositions(p)

main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !
main vide !


In [427]:
a = NotesAutomata(["do", "ré", "mi"], 5, 2)

p = music_to_seq(a, [[("do", 1), ("ré", 1), ("mi", 1)], 
                     [("do", 1)], 
                     [("do", 1)],
                     [("do", 1)],
                     [("ré", 4)],
                     [("mi", 5)],
                     [("ré", 3)]])
pp_propositions(p)

TypeError: music_to_seq() missing 1 required positional argument: 'max_all_None'

In [107]:
a = NotesAutomata(["do", "ré", "mi"], 5, 2)

 
#                  [("mi", 5)],
#                  [("ré", 2)],
#                  [("do", 4)],
#                  [("mi", 3)],
#                  [("ré", 3)],
#                  [("ré", 1)],
#                  [("do", 4)]

p = music_to_seq(a, [[("do", 1), ("ré", 1), ("mi", 1)], 
                     [("do", 1)], 
                     [("ré", 2), ("mi", 2), ("do", 1)],
                     [("do", 1)]])
pp_propositions(p)

In [423]:
p[((('do', 'ré', 'mi'), (), (), (), ()), ((), (), (), (), ()))]

[[((((2, 'ré'), (2, 'mi')), ((1, 'do'),)), ((), ())),
  (((), ()), (((1, 'do'),), ())),
  ((((None, 'ré'), (None, 'mi'), (1, 'do')), ()), ((), ()))],
 [((((2, 'ré'), (2, 'mi')), ((1, 'do'),)), ((), ())),
  (((), ()), (((1, 'do'),), ())),
  ((((None, 'ré'), (None, 'mi')), ((1, 'do'),)), ((), ()))],
 [((((2, 'ré'), (2, 'mi')), ((1, 'do'),)), ((), ())),
  (((), ()), ((), ((1, 'do'),))),
  ((((None, 'ré'), (None, 'mi')), ()), (((1, 'do'),), ()))],
 [((((2, 'ré'), (2, 'mi')), ((1, 'do'),)), ((), ())),
  (((), ()), ((), ((1, 'do'),))),
  ((((None, 'ré'), (None, 'mi')), ()), ((), ((1, 'do'),)))],
 [((((1, 'do'), (None, 'ré')), ((2, 'mi'),)), ((), ())),
  ((((1, 'ré'),), ((1, 'do'),)), ((), ())),
  ((((None, 'ré'),), ()), ((), ((None, 'mi'), (1, 'do'))))],
 [((((1, 'do'), (None, 'ré')), ((2, 'mi'),)), ((), ())),
  ((((1, 'ré'),), ((1, 'do'),)), ((), ())),
  ((((None, 'ré'),), ()), (((1, 'do'),), ((None, 'mi'),)))],
 [((((1, 'do'), (None, 'ré')), ((2, 'mi'),)), ((), ())),
  (((), ((1, 'do'), (1

In [145]:
trs = a.possible_transitions([[["do", "ré", "mi"], [], [], [], []], [[], [], [], [], []]])

for t, s in trs:
    for sh in t:
        for dh in sh:
            if len(dh) == 3:
                if (1, "do") in dh and (2, "ré") in dh:
                    print(t, s)

((((1, 'do'), (2, 'ré'), (1, 'mi')), ()), ((), ())) [[['do', 'mi'], ['ré'], [], [], []], [[], [], [], [], []]]
(((), ((1, 'do'), (2, 'ré'), (2, 'mi'))), ((), ())) [[[], [], [], [], []], [['do'], ['ré', 'mi'], [], [], []]]
((((1, 'do'), (2, 'ré'), (2, 'mi')), ()), ((), ())) [[['do'], ['ré', 'mi'], [], [], []], [[], [], [], [], []]]
((((1, 'do'), (2, 'ré'), (5, 'mi')), ()), ((), ())) [[['do'], ['ré'], [], [], ['mi']], [[], [], [], [], []]]
((((1, 'do'), (2, 'ré'), (3, 'mi')), ()), ((), ())) [[['do'], ['ré'], ['mi'], [], []], [[], [], [], [], []]]
(((), ((1, 'do'), (2, 'ré'), (3, 'mi'))), ((), ())) [[[], [], [], [], []], [['do'], ['ré'], ['mi'], [], []]]
(((), ((1, 'do'), (2, 'ré'), (4, 'mi'))), ((), ())) [[[], [], [], [], []], [['do'], ['ré'], [], ['mi'], []]]
(((), ((1, 'do'), (2, 'ré'), (5, 'mi'))), ((), ())) [[[], [], [], [], []], [['do'], ['ré'], [], [], ['mi']]]
((((1, 'do'), (2, 'ré'), (4, 'mi')), ()), ((), ())) [[['do'], ['ré'], [], ['mi'], []], [[], [], [], [], []]]
(((), ((1, 'd

In [129]:
((((2, "mi"), (1, "do"), (2, "ré"),), ()), ((), ())) in trs

False

In [148]:
seq_to_music(a, 
             [[["do", "ré", "mi"], [], [], [], []], [[], [], [], [], []]],
             [
                 ((((1, "do"), (2, "ré"), (2, "mi"),), ()), ((), ())),
                 ((((1, "do"),), ()), ((), ()))
             ])

[[('do', 1), ('ré', 1), ('mi', 1)],
 [('do', 1)],
 [('ré', 2), ('mi', 2), ('do', 1)]]

In [472]:
music_to_seq2(NotesAutomata(["do", "ré", "mi"], 9), [(2, "do"), (2, "do"), (2, "do"), (2, "ré")])

NameError: name 'music_to_seq2' is not defined

In [36]:
a = NotesAutomata(["do", "ré", "mi"], 9)

In [37]:
a.transition(["do", "ré", "mi", None, None, None, None, None, None], 4)

['ré', 'mi', None, 'do', None, None, None, None, None]

In [38]:
a.transition(['ré', 'mi', None, 'do', None, None, None, None, False], 9)

['mi', None, 'do', None, None, None, None, False, 'ré']

In [39]:
a.transition(['mi', None, 'do', None, None, None, None, False, 'ré'], 9)

[None, 'do', None, None, None, None, False, 'ré', 'mi']

In [40]:
music_to_seq2(NotesAutomata(["do", "ré", "mi"], 12), [(2, "do"), (2, "do"), (2, "do"), (2, "ré"), (2, "mi"), (4, "ré"), (4, "do"), (2, "mi"), (2, "ré"), (2, "ré"), (2, "do")])

{'0-do-0-0-0-0-0-ré-0-mi-0-0': [[0,
   2,
   0,
   2,
   0,
   12,
   0,
   6,
   0,
   10,
   0,
   0,
   0,
   8,
   0,
   0,
   0,
   8,
   0,
   7,
   0,
   2,
   0,
   4,
   0],
  [0,
   2,
   0,
   2,
   0,
   12,
   0,
   6,
   0,
   10,
   0,
   0,
   0,
   8,
   0,
   0,
   0,
   8,
   0,
   7,
   0,
   2,
   0,
   5,
   0],
  [0,
   2,
   0,
   2,
   0,
   12,
   0,
   6,
   0,
   10,
   0,
   0,
   0,
   8,
   0,
   0,
   0,
   8,
   0,
   7,
   0,
   2,
   0,
   6,
   0],
  [0,
   2,
   0,
   2,
   0,
   12,
   0,
   6,
   0,
   10,
   0,
   0,
   0,
   8,
   0,
   0,
   0,
   8,
   0,
   7,
   0,
   2,
   0,
   7,
   0],
  [0,
   2,
   0,
   2,
   0,
   12,
   0,
   6,
   0,
   10,
   0,
   0,
   0,
   8,
   0,
   0,
   0,
   8,
   0,
   7,
   0,
   2,
   0,
   8,
   0],
  [0,
   2,
   0,
   2,
   0,
   12,
   0,
   6,
   0,
   10,
   0,
   0,
   0,
   8,
   0,
   0,
   0,
   8,
   0,
   7,
   0,
   2,
   0,
   9,
   0],
  [0,
   2,
   0,
   2,
   0,
   12,
   0,
   6,
   

In [41]:
a = NotesAutomata(["do", "ré", "mi"], 12)
s1 = ["do", "ré", "mi"] + [None for i in range(3, 12)]
s2 = a.transition(s1, 4)
s3 = a.transition(s2, 9)
s4 = a.transition(s3, 10)
a.state_str(s4)

'0-do-0-0-0-0-0-ré-0-mi-0-0'